In [ ]:
'''#数据准备 资源限制区间数据和用户参数数据
#从excle表generator-abcPmaxPmin.xlsx中提取出相关数据
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
import xlrd
import xlwt

book = xlrd.open_workbook('generator-abcPmaxPmin.xlsx')
sheet1 = book.sheets()[0]
nrows = sheet1.nrows
ncols = sheet1.ncols
generator_a_values = sheet1.col_values(2)
generator_b_values = sheet1.col_values(3)
generator_c_values = sheet1.col_values(4)
generator_Pmax_values = sheet1.col_values(5)
generator_Pmin_values = sheet1.col_values(6)'''

In [ ]:
'''#数据存储
np.savetxt('./generator_a_values.txt',generator_a_values,fmt='%0.8f')
np.savetxt('./generator_b_values.txt',generator_b_values,fmt='%0.8f')
np.savetxt('./generator_c_values.txt',generator_c_values,fmt='%0.8f')
np.savetxt('./generator_Pmax_values.txt',generator_Pmax_values,fmt='%0.8f')
np.savetxt('./generator_Pmin_values.txt',generator_Pmin_values,fmt='%0.8f')'''

In [ ]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 

# DRA算法+无拜占庭攻击

In [ ]:
#定义变量
agent_resource_quantity = np.zeros(shape=(54,2000)) #54个agent的资源量 定义成一个矩阵 54*1000 
average_resource_quantity_true= [] #54个agent的平均资源量
x_average_resource_quantity=np.zeros(shape=(54,2000)) #动态一致平均算法追踪平均值的变量
v=0#v-强凸参数 
iteration_update = 0.5#迭代步长
generator_a_values= np.loadtxt("generator_a_values.txt")#每个agent的成本函数中的超参数a
generator_b_values= np.loadtxt("generator_b_values.txt")#每个agent的成本函数中的超参数a
generator_c_values= np.loadtxt("generator_c_values.txt")#每个agent的成本函数中的超参数a
generator_Pmax_values= np.loadtxt("generator_Pmax_values.txt")#每个agent的资源量的上界
generator_Pmin_values= np.loadtxt("generator_Pmin_values.txt")#每个agent的资源量的下界
dual_variable = np.zeros(shape=(54,2000))#对偶变量
W=np.loadtxt("Adjacency_Matrix_generator_doubly_stochastic.txt") #双随机的网络拓扑矩阵W（对角线处非零）
sum = 0 #过渡变量
iteration=[]#绘图横坐标
average_cost=np.zeros(shape=(1,2000))#存放每一次迭代后所有用户成本的平均值
x_average=[]#存放每一次迭代后所有用户追踪平均值x的平均值

In [ ]:
#变量初始化
#用户资源量的初始化
for i in range(0,54):
    agent_resource_quantity[i][0]=50
        
#追踪均值变量的初始化 x(0)=agent_resource_quantity(0)
for i in range (0,54):
    x_average_resource_quantity[i][0]=agent_resource_quantity[i][0]
    
#资源量真实均值的计算 初始值
sum1=0
for i in range (0,54):
    sum1 = sum1 + agent_resource_quantity[i][0] 
average_resource_quantity_true.append(sum1/54)

#平均追踪值x的平均值 初始值
sum2=0
for i in range (0,54):
    sum2 = sum2 + x_average_resource_quantity[i][0] 
x_average.append(sum2/54)

#平均成本 初始值
sum_average_cost=0
for j in range(0,54):
    sum_average_cost=sum_average_cost+(generator_a_values[j]+(generator_b_values[j]*agent_resource_quantity[j][0])+(generator_b_values[j]*agent_resource_quantity[j][0]*agent_resource_quantity[j][0]))
average_cost[0][0]=sum_average_cost/54
    
#对偶变量的初始化
for i in range (0,54):
    dual_variable[i][0]=1

iteration.append(0)

In [ ]:
#FODAC-PD-DRA算法 
def FODAC_PD_DRA(generator_a_values,generator_b_values,generator_c_values,generator_Pmax_values,generator_Pmin_values,agent_resource_quantity,average_resource_quantity_true,x_average_resource_quantity,v,iteration_update,dual_variable,W,iteration):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DRA算法+无拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,54):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity[i][0])
    print("*************************")
    print("初始资源申请量真实平均值avg为:",average_resource_quantity_true[0])
    print("*************************")
    print("平均资源追踪量的初始平均值avg为:",x_average[0])
    print("*************************")
    for i in range(0,54):
        print("agent",(i+1),"的初始追踪平均量为:",x_average_resource_quantity[i][0])
    print("*************************")
    for i in range(0,54):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update)
    print("****************************************************************************************************")
    
    #开始迭代
    for k in range(1,2000):#迭代次数
        sum1 = 0
        sum2 = 0
        sum_average_cost=0
        iteration.append(k)
        
        #用户资源申请量的迭代
        for i in range (0,54):
            agent_resource_quantity[i][k]=(agent_resource_quantity[i][k-1]-iteration_update
            *(((generator_b_values[i]+2*generator_c_values[i]*agent_resource_quantity[i][k-1])/54)+(dual_variable[i][k-1]/54)))
            #原变量的投影操作
            if agent_resource_quantity[i][k] < generator_Pmin_values[i]:
                agent_resource_quantity[i][k]= generator_Pmin_values[i]
            elif agent_resource_quantity[i][k] > generator_Pmax_values[i]:
                agent_resource_quantity[i][k]=generator_Pmax_values[i]
                
        #平均值追踪量的迭代 最新版 Wx+资源量差值
        for i in range (0,54):
            sum_x=0
            for j in range (0,54):
                if W[i][j]!=0:
                    sum_x=sum_x+W[i][j]*x_average_resource_quantity[j][k-1]
            x_average_resource_quantity[i][k]=sum_x+(agent_resource_quantity[i][k]-agent_resource_quantity[i][k-1])        
        
        #对偶变量的迭代
        for i in range(0,54):
            dual_variable[i][k]=(dual_variable[i][k-1]+iteration_update
            *((x_average_resource_quantity[i][k-1])-(6000/54)-(v*dual_variable[i][k-1])))
        
        #对偶变量的正投影操作
        '''for i in range(0,54):
            if dual_variable[i][k] <0:
                dual_variable[i][k] =0 '''   
        
        #计算真实的平均资源
        for i in range (0,54):
            sum1 = sum1 + agent_resource_quantity[i][k]
        average_resource_quantity_true.append(sum1/54)
        
        #计算平均资源追踪值x的平均值
        for i in range (0,54):
            sum2 = sum2 + x_average_resource_quantity[i][k]
        x_average.append(sum2/54)
        
        #计算所有用户的平均成本
        for j in range(0,54):
            sum_average_cost=sum_average_cost+(generator_a_values[j]+(generator_b_values[j]*agent_resource_quantity[j][k])+(generator_b_values[j]*agent_resource_quantity[j][k]*agent_resource_quantity[j][k]))
        average_cost[0][k]=sum_average_cost/54   
        
        #输出第k次迭代中的结果
        print(k,"次迭代中的结果：")
        for i in range (0,54):
            print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity[i][k])
        print("资源申请量真实平均值avg为:",average_resource_quantity_true[k])
        print("****************************************************************************")
        print("平均资源追踪量的平均值avg为:",x_average[k])
        for i in range(0,54):
            print("agent",(i+1),"在",k,"次迭代中的的追踪平均量为:",x_average_resource_quantity[i][k])
        for i in range(0,54):
            print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable[i][k])       
        print("****************************************************************************")
    
    return None

In [ ]:
FODAC_PD_DRA(generator_a_values,generator_b_values,generator_c_values,generator_Pmax_values,generator_Pmin_values,agent_resource_quantity,average_resource_quantity_true,x_average_resource_quantity,v,iteration_update,dual_variable,W,iteration)

In [ ]:
#数据存储
np.savetxt('./data/05_agent_resource_quantity.txt',agent_resource_quantity,fmt='%0.8f')#用户资源
np.savetxt('./data/05_average_cost.txt',average_cost,fmt='%0.8f')#用户平均成本
np.savetxt('./data/05_average_resource_quantity_true.txt',average_resource_quantity_true,fmt='%0.8f')#用户真实平均资源

# DRA算法+极大值攻击

In [ ]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 

In [ ]:
#定义变量 
agent_resource_quantity_attack1 = np.zeros(shape=(54,2000)) 
average_resource_quantity_true_attack1= [] 
x_average_resource_quantity_attack1=np.zeros(shape=(54,2000)) 
v_attack1=0
iteration_update_attack1 = 0.5
dual_variable_attack1 = np.zeros(shape=(54,2000))#对偶变量
generator_a_values_attack1= np.loadtxt("generator_a_values.txt")#每个agent的成本函数中的超参数a
generator_b_values_attack1= np.loadtxt("generator_b_values.txt")#每个agent的成本函数中的超参数a
generator_c_values_attack1= np.loadtxt("generator_c_values.txt")#每个agent的成本函数中的超参数a
generator_Pmax_values_attack1= np.loadtxt("generator_Pmax_values.txt")#每个agent的资源量的上界
generator_Pmin_values_attack1= np.loadtxt("generator_Pmin_values.txt")#每个agent的资源量的下界
W_attack1=np.loadtxt("Adjacency_Matrix_generator_doubly_stochastic.txt") #双随机的网络拓扑矩阵W（对角线处非零）
number_attack1 = 1 #攻击者数量
index_attack1=[27]#攻击者的索引 下标
sum_attack1 = 0 #过渡变量
iteration_attack1=[]#绘图横坐标
average_cost_attack1=np.zeros(shape=(1,2000))#存放每一次迭代后所有用户成本的平均值
average_cost_honest_attack1=np.zeros(shape=(1,2000))#存放每一次迭代后所有诚实用户成本的平均值
average_honest_resource_quantity_attack1=[]#存放每一次迭代后诚实用户的平均资源量
x_average_attack1=[]#存放每一次迭代后所有用户追踪平均值x的平均值

In [ ]:
#变量初始化
#用户资源量的初始化
for i in range(0,54):
    agent_resource_quantity_attack1[i][0]=50
    
#设置攻击者的初始攻击
agent_resource_quantity_attack1[index_attack1[0]][0]=420
        
#追踪均值变量的初始化 x(0)=agent_resource_quantity(0)
for i in range (0,54):
    x_average_resource_quantity_attack1[i][0]=agent_resource_quantity_attack1[i][0]
    
#资源量真实均值的计算 初始值
sum1_attack1=0
for i in range (0,54):
    sum1_attack1 = sum1_attack1 + agent_resource_quantity_attack1[i][0] 
average_resource_quantity_true_attack1.append(sum1_attack1/54)

#平均追踪值x的平均值 初始值
sum2_attack1=0
for i in range (0,54):
    sum2_attack1 = sum2_attack1 + x_average_resource_quantity_attack1[i][0] 
x_average_attack1.append(sum2_attack1/54)

#平均成本 初始值
sum_average_cost_attack1=0
for j in range(0,54):
    sum_average_cost_attack1=sum_average_cost_attack1+(generator_a_values_attack1[j]+(generator_b_values_attack1[j]*agent_resource_quantity_attack1[j][0])+(generator_b_values_attack1[j]*agent_resource_quantity_attack1[j][0]*agent_resource_quantity_attack1[j][0]))
average_cost_attack1[0][0]=sum_average_cost_attack1/54
    
#对偶变量的初始化
for i in range (0,54):
    dual_variable_attack1[i][0]=1

iteration_attack1.append(0)

#计算诚实用户的平均成本 初始值 
sum_average_cost_honest_attack1=0
for j in range(0,54):
    if j!=index_attack1[0]:
        sum_average_cost_honest_attack1=(sum_average_cost_honest_attack1
    +(generator_a_values_attack1[j]+(generator_b_values_attack1[j]*agent_resource_quantity_attack1[j][0])+(generator_b_values_attack1[j]*agent_resource_quantity_attack1[j][0]*agent_resource_quantity_attack1[j][0])))        
average_cost_honest_attack1[0][0]=sum_average_cost_honest_attack1/(54-number_attack1)

#诚实用户的平均资源量
sum_honest_resource_quantity_attack1=0
for j in range(0,54):
    if j!=index_attack1[0]:
        sum_honest_resource_quantity_attack1=sum_honest_resource_quantity_attack1+agent_resource_quantity_attack1[j][0]
average_honest_resource_quantity_attack1.append(sum_honest_resource_quantity_attack1/(54-number_attack1))

In [ ]:
#FODAC-PD-DRA_Static_Byzantine_attack1算法-确定性攻击-攻击方式：被攻击的agent在每次迭代中始终保持相同的恶意资源申请量（不进行更新），并将该恶意资源量发送给邻居
def FODAC_PD_DRA_Static_Byzantine_attack1(generator_a_values_attack1,generator_b_values_attack1,generator_c_values_attack1,generator_Pmax_values_attack1,generator_Pmin_values_attack1,agent_resource_quantity_attack1,average_resource_quantity_true_attack1,x_average_resource_quantity_attack1,v_attack1,iteration_update_attack1,dual_variable_attack1,W_attack1,iteration_attack1):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DRA算法+极大值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,54):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_attack1[i][0])
    print("*************************")
    print("初始资源申请量真实平均值avg为:",average_resource_quantity_true_attack1[0])
    print("*************************")
    print("平均资源追踪量的初始平均值avg为:",x_average_attack1[0])
    print("*************************")
    for i in range(0,54):
        print("agent",(i+1),"的初始追踪平均量为:",x_average_resource_quantity_attack1[i][0])
    print("*************************")
    for i in range(0,54):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_attack1[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_attack1)
    print("****************************************************************************************************")
    
    #开始迭代
    for k in range(1,2000):#迭代次数
        sum1_attack1 = 0
        sum2_attack1 = 0
        sum_average_cost_attack1=0
        sum_average_cost_honest_attack1=0
        iteration_attack1.append(k)
        sum_honest_resource_quantity_attack1=0
        
        #计算诚实用户和攻击用户的资源申请量迭代
        for i in range (0,54):
            if i!=index_attack1[0]:#诚实用户
                agent_resource_quantity_attack1[i][k]=(agent_resource_quantity_attack1[i][k-1]-iteration_update_attack1
            *(((generator_b_values_attack1[i]+2*generator_c_values_attack1[i]*agent_resource_quantity_attack1[i][k-1])/54)+(dual_variable_attack1[i][k-1]/54)))
                #原变量的投影操作
                if agent_resource_quantity_attack1[i][k] < generator_Pmin_values_attack1[i]:
                    agent_resource_quantity_attack1[i][k]= generator_Pmin_values_attack1[i]
                elif agent_resource_quantity_attack1[i][k] > generator_Pmax_values_attack1[i]:
                    agent_resource_quantity_attack1[i][k]=generator_Pmax_values_attack1[i]
            else:#攻击用户
                agent_resource_quantity_attack1[i][k]=agent_resource_quantity_attack1[i][k-1]
            
        #计算诚实用户和攻击用户的平均值追踪量的迭代 Wx+资源量差值
        for i in range (0,54):
            if i!=index_attack1[0]:#诚实用户
                sum_x_attack1=0
                for j in range (0,54):
                    if W_attack1[i][j]!=0:
                        sum_x_attack1=sum_x_attack1+W_attack1[i][j]*x_average_resource_quantity_attack1[j][k-1]
                x_average_resource_quantity_attack1[i][k]=sum_x_attack1+(agent_resource_quantity_attack1[i][k]-agent_resource_quantity_attack1[i][k-1])                     
            else:#攻击者
                x_average_resource_quantity_attack1[i][k]=agent_resource_quantity_attack1[i][k-1] 
        
        #诚实用户和攻击者对偶变量的迭代
        for i in range(0,54):
            if i!=index_attack1[0]:#诚实用户
                dual_variable_attack1[i][k]=(dual_variable_attack1[i][k-1]+iteration_update_attack1
                *((x_average_resource_quantity_attack1[i][k-1])-(6000/54)-(v_attack1*dual_variable_attack1[i][k-1])))
            else:#攻击者
                dual_variable_attack1[i][k]=dual_variable_attack1[i][k-1] #被攻击用户的对偶变量不进行更新
            
        #对偶变量的正投影操作
        '''for i in range(0,100-int(proportion_attack1*100)):
            if dual_variable_attack1[i][k] < 0:
                dual_variable_attack1[i][k] =0 '''
                
        #计算所有用户平均资源量
        for i in range (0,54):
            sum1_attack1 = sum1_attack1 + agent_resource_quantity_attack1[i][k]
        average_resource_quantity_true_attack1.append(sum1_attack1/54)
        
        #计算平均资源追踪值x的平均值
        for i in range (0,54):
            sum2_attack1 = sum2_attack1 + x_average_resource_quantity_attack1[i][k]
        x_average_attack1.append(sum2_attack1/54)
        
        #计算所有用户平均成本
        for j in range(0,54):
            sum_average_cost_attack1=(sum_average_cost_attack1+
            (generator_a_values_attack1[j]+(generator_b_values_attack1[j]*agent_resource_quantity_attack1[j][k])
             +(generator_b_values_attack1[j]*agent_resource_quantity_attack1[j][k]*agent_resource_quantity_attack1[j][k])))
        average_cost_attack1[0][k]=sum_average_cost_attack1/54

        #计算诚实用户的平均成本 
        for j in range(0,54):
            if j!=index_attack1[0]:#诚实用户
                sum_average_cost_honest_attack1=(sum_average_cost_honest_attack1+(generator_a_values_attack1[j]+(generator_b_values_attack1[j]*agent_resource_quantity_attack1[j][k])+(generator_b_values_attack1[j]*agent_resource_quantity_attack1[j][k]*agent_resource_quantity_attack1[j][k])))    
        average_cost_honest_attack1[0][k]=sum_average_cost_honest_attack1/(54-number_attack1)
        
        #计算诚实用户的平均资源
        for j in range(0,54):
            if j!=index_attack1[0]:#诚实用户
                sum_honest_resource_quantity_attack1=sum_honest_resource_quantity_attack1+agent_resource_quantity_attack1[j][k]        
        average_honest_resource_quantity_attack1.append(sum_honest_resource_quantity_attack1/(54-number_attack1))
        
        #输出第k次迭代中的结果
        print(k,"次迭代中的结果：")
        for i in range (0,54):
            print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_attack1[i][k])
        print("资源申请量真实平均值avg为:",average_resource_quantity_true_attack1[k])
        print("****************************************************************************")
        print("平均资源追踪量的平均值avg为:",x_average_attack1[k])
        for i in range(0,54):
            print("agent",(i+1),"在",k,"次迭代中的的追踪平均量为:",x_average_resource_quantity_attack1[i][k])
        for i in range(0,54):
            print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_attack1[i][k])       
        print("****************************************************************************")
    
    return None

In [ ]:
FODAC_PD_DRA_Static_Byzantine_attack1(generator_a_values_attack1,generator_b_values_attack1,generator_c_values_attack1,generator_Pmax_values_attack1,generator_Pmin_values_attack1,agent_resource_quantity_attack1,average_resource_quantity_true_attack1,x_average_resource_quantity_attack1,v_attack1,iteration_update_attack1,dual_variable_attack1,W_attack1,iteration_attack1)

In [ ]:
#数据存储
np.savetxt('./data/05_agent_resource_quantity_attack1.txt',agent_resource_quantity_attack1,fmt='%0.8f')#用户资源
np.savetxt('./data/05_average_cost_honest_attack1.txt',average_cost_honest_attack1,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/05_average_honest_resource_quantity_attack1.txt',average_honest_resource_quantity_attack1,fmt='%0.8f')#诚实用户平均资源

# BREDA算法 +极大值攻击

In [ ]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 

In [ ]:
#定义变量 
agent_resource_quantity_robust1= np.zeros(shape=(54,1000)) 
average_resource_quantity_true_robust1= [] 
x_average_resource_quantity_robust1=np.zeros(shape=(54,1000)) 
v_robust1=0
iteration_update_robust1 = 0.5
dual_variable_robust1 = np.zeros(shape=(54,1000))#对偶变量
generator_a_values_robust1= np.loadtxt("generator_a_values.txt")#每个agent的成本函数中的超参数a
generator_b_values_robust1= np.loadtxt("generator_b_values.txt")#每个agent的成本函数中的超参数a
generator_c_values_robust1= np.loadtxt("generator_c_values.txt")#每个agent的成本函数中的超参数a
generator_Pmax_values_robust1= np.loadtxt("generator_Pmax_values.txt")#每个agent的资源量的上界
generator_Pmin_values_robust1= np.loadtxt("generator_Pmin_values.txt")#每个agent的资源量的下界
W_robust1=np.loadtxt("Adjacency_Matrix_generator_doubly_stochastic.txt") #双随机的网络拓扑矩阵W（对角线处非零）
A_robust1=np.loadtxt("Adjacency_Matrix_generator_zero.txt") #网络拓扑邻接矩阵（对角线处为0）
D_robust1=np.loadtxt("degree_generator.txt") #用户的度
number_robust1 = 1 #攻击者数量
index_robust1=[27]#攻击者的索引 下标
sum_robust1 = 0 #过渡变量
iteration_robust1=[]#绘图横坐标
average_cost_robust1=np.zeros(shape=(1,1000))#存放每一次迭代后所有用户成本的平均值
average_cost_honest_robust1=np.zeros(shape=(1,1000))#存放每一次迭代后所有诚实用户成本的平均值
average_honest_resource_quantity_robust1=[]#存放每一次迭代后诚实用户的平均资源量
x_average_robust1=[]#存放每一次迭代后所有用户追踪平均值x的平均值
neighbor_robust1=np.zeros(shape=(54,16))#定义矩阵存放54个用户的邻居
neighbor_robust1_help=[]#辅助求用户邻居的列表

In [ ]:
#变量初始化
#用户资源量的初始化
for i in range(0,54):
    agent_resource_quantity_robust1[i][0]=50
    
#设置攻击者的初始攻击
agent_resource_quantity_robust1[index_robust1[0]][0]=420
        
#追踪均值变量的初始化 x(0)=agent_resource_quantity(0)
for i in range (0,54):
    x_average_resource_quantity_robust1[i][0]=agent_resource_quantity_robust1[i][0]
    
#资源量真实均值的计算 初始值
sum1_robust1=0
for i in range (0,54):
    sum1_robust1 = sum1_robust1 + agent_resource_quantity_robust1[i][0] 
average_resource_quantity_true_robust1.append(sum1_robust1/54)

#平均追踪值x的平均值 初始值
sum2_robust1=0
for i in range (0,54):
    sum2_robust1 = sum2_robust1 + x_average_resource_quantity_robust1[i][0] 
x_average_robust1.append(sum2_robust1/54)

#平均成本 初始值
sum_average_cost_robust1=0
for j in range(0,54):
    sum_average_cost_robust1=sum_average_cost_robust1+(generator_a_values_robust1[j]+(generator_b_values_robust1[j]*agent_resource_quantity_robust1[j][0])+(generator_b_values_robust1[j]*agent_resource_quantity_robust1[j][0]*agent_resource_quantity_robust1[j][0]))
average_cost_robust1[0][0]=sum_average_cost_robust1/54
    
#对偶变量的初始化
for i in range (0,54):
    dual_variable_robust1[i][0]=1

iteration_robust1.append(0)

#计算诚实用户的平均成本 初始值 
sum_average_cost_honest_robust1=0
for j in range(0,54):
    if j!=index_robust1[0]:
        sum_average_cost_honest_robust1=(sum_average_cost_honest_robust1
    +(generator_a_values_robust1[j]+(generator_b_values_robust1[j]*agent_resource_quantity_robust1[j][0])+(generator_b_values_robust1[j]*agent_resource_quantity_robust1[j][0]*agent_resource_quantity_robust1[j][0])))        
average_cost_honest_robust1[0][0]=sum_average_cost_honest_robust1/(54-number_robust1)

#诚实用户的平均资源量
sum_honest_resource_quantity_robust1=0
for j in range(0,54):
    if j!=index_robust1[0]:
        sum_honest_resource_quantity_robust1=sum_honest_resource_quantity_robust1+agent_resource_quantity_robust1[j][0]
average_honest_resource_quantity_robust1.append(sum_honest_resource_quantity_robust1/(54-number_robust1))

#对用户邻居进行初始化
for i in range(0,54):
    for j in range(0,16):
        neighbor_robust1[i][j]=99 #先将矩阵中的所有数字设定为一个不可能的数字
#根据邻接矩阵求每个用户的邻居集合
for i in range(0,54):
    for j in range(0,54):
        if A_robust1[i][j]!=0:
            neighbor_robust1_help.append(j)
k=0
for i in range(0,54):
    g=0
    for j in range(k,k+int(D_robust1[i])):
        neighbor_robust1[i][g]=neighbor_robust1_help[j]  
        g=g+1
    k=k+int(D_robust1[i])    

In [ ]:
#BREDA算法 用TM进行x的鲁棒聚合
def FODAC_PD_DRA_Static_Byzantine_robust1(generator_a_values_robust1,generator_b_values_robust1,generator_c_values_robust1,generator_Pmax_values_robust1,generator_Pmin_values_robust1,agent_resource_quantity_robust1,average_resource_quantity_true_robust1,x_average_resource_quantity_robust1,v_robust1,iteration_update_robust1,dual_variable_robust1,W_robust1,iteration_robust1):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************BREDA算法+极大值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,54):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust1[i][0])
    print("*************************")
    print("初始资源申请量真实平均值avg为:",average_resource_quantity_true_robust1[0])
    print("*************************")
    print("平均资源追踪量的初始平均值avg为:",x_average_robust1[0])
    print("*************************")
    for i in range(0,54):
        print("agent",(i+1),"的初始追踪平均量为:",x_average_resource_quantity_robust1[i][0])
    print("*************************")
    for i in range(0,54):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust1[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust1)
    print("****************************************************************************************************")
    
    #开始迭代
    for k in range(1,1000):#迭代次数
        sum1_robust1 = 0
        sum2_robust1 = 0
        sum_average_cost_robust1=0
        sum_average_cost_honest_robust1=0
        iteration_robust1.append(k)
        sum_honest_resource_quantity_robust1=0
        
        #计算诚实用户和攻击用户的资源申请量迭代
        for i in range (0,54):
            if i!=index_robust1[0]:#诚实用户
                agent_resource_quantity_robust1[i][k]=(agent_resource_quantity_robust1[i][k-1]-iteration_update_robust1
            *(((generator_b_values_robust1[i]+2*generator_c_values_robust1[i]*agent_resource_quantity_robust1[i][k-1])/54)+(dual_variable_robust1[i][k-1]/54)))
                #原变量的投影操作
                if agent_resource_quantity_robust1[i][k] < generator_Pmin_values_robust1[i]:
                    agent_resource_quantity_robust1[i][k]= generator_Pmin_values_robust1[i]
                elif agent_resource_quantity_robust1[i][k] > generator_Pmax_values_robust1[i]:
                    agent_resource_quantity_robust1[i][k]=generator_Pmax_values_robust1[i]
            else:#攻击用户
                agent_resource_quantity_robust1[i][k]=agent_resource_quantity_robust1[i][k-1]
            
        #计算诚实用户和攻击用户的平均值追踪量的迭代 Wx+资源量差值
        #用trimmed-mean算法对诚实用户的x进行鲁棒聚合
        for i in range (0,54):
            if i!=index_robust1[0]:#诚实用户
                sort_list_robust1=[]
                sum_x_robust1=0
                for j in range(0,int(D_robust1[i])):
                    sort_list_robust1.append(x_average_resource_quantity_robust1[int(neighbor_robust1[i][j])][k-1])
                sort_list_robust1.sort()
                #剔除一个最大的数据和一个最小的数据
                if int(D_robust1[i])>2: #度大于2的用户
                    del sort_list_robust1[int(D_robust1[i])-1]
                    del sort_list_robust1[0]
                    for j in range(0,len(sort_list_robust1)):
                        sum_x_robust1=sum_x_robust1+sort_list_robust1[j]
                    x_average_resource_quantity_robust1[i][k]=((1/(len(sort_list_robust1)+1))*(x_average_resource_quantity_robust1[i][k-1]+sum_x_robust1)
                    +(agent_resource_quantity_robust1[i][k]-agent_resource_quantity_robust1[i][k-1]))
                if int(D_robust1[i])==2: #对于度小于等于2的用户 不进行TM操作 直接用WX
                    for j in range(0,2):
                        sum_x_robust1=sum_x_robust1+(W_robust1[i][int(neighbor_robust1[i][j])]*x_average_resource_quantity_robust1[int(neighbor_robust1[i][j])][k-1])                       
                    x_average_resource_quantity_robust1[i][k]=(W_robust1[i][i]*x_average_resource_quantity_robust1[i][k-1]+sum_x_robust1
                    +(agent_resource_quantity_robust1[i][k]-agent_resource_quantity_robust1[i][k-1]))
                if int(D_robust1[i])==1:
                    x_average_resource_quantity_robust1[i][k]=(W_robust1[i][i]*x_average_resource_quantity_robust1[i][k-1]+W_robust1[i][int(neighbor_robust1[i][j])]*x_average_resource_quantity_robust1[int(neighbor_robust1[i][0])][k-1]
                    +(agent_resource_quantity_robust1[i][k]-agent_resource_quantity_robust1[i][k-1]))                                
            else:#攻击者
                x_average_resource_quantity_robust1[i][k]=agent_resource_quantity_robust1[i][k-1]
        
        #诚实用户和攻击者对偶变量的迭代
        for i in range(0,54):
            if i!=index_robust1[0]:#诚实用户
                dual_variable_robust1[i][k]=(dual_variable_robust1[i][k-1]+iteration_update_robust1
                *((x_average_resource_quantity_robust1[i][k-1])-(6000/54)-(v_robust1*dual_variable_robust1[i][k-1])))
            else:#攻击者
                dual_variable_robust1[i][k]=dual_variable_robust1[i][k-1] #被攻击用户的对偶变量不进行更新
            
        #对偶变量的正投影操作
        '''for i in range(0,100-int(proportion_robust1*100)):
            if dual_variable_robust1[i][k] < 0:
                dual_variable_robust1[i][k] =0 '''
                
        #计算所有用户平均资源量
        for i in range (0,54):
            sum1_robust1 = sum1_robust1 + agent_resource_quantity_robust1[i][k]
        average_resource_quantity_true_robust1.append(sum1_robust1/54)
        
        #计算平均资源追踪值x的平均值
        for i in range (0,54):
            sum2_robust1 = sum2_robust1 + x_average_resource_quantity_robust1[i][k]
        x_average_robust1.append(sum2_robust1/54)
        
        #计算所有用户平均成本
        for j in range(0,54):
            sum_average_cost_robust1=(sum_average_cost_robust1+
            (generator_a_values_robust1[j]+(generator_b_values_robust1[j]*agent_resource_quantity_robust1[j][k])
             +(generator_b_values_robust1[j]*agent_resource_quantity_robust1[j][k]*agent_resource_quantity_robust1[j][k])))
        average_cost_robust1[0][k]=sum_average_cost_robust1/54

        #计算诚实用户的平均成本 
        for j in range(0,54):
            if j!=index_robust1[0]:#诚实用户
                sum_average_cost_honest_robust1=(sum_average_cost_honest_robust1+(generator_a_values_robust1[j]+(generator_b_values_robust1[j]*agent_resource_quantity_robust1[j][k])+(generator_b_values_robust1[j]*agent_resource_quantity_robust1[j][k]*agent_resource_quantity_robust1[j][k])))    
        average_cost_honest_robust1[0][k]=sum_average_cost_honest_robust1/(54-number_robust1)
        
        #计算诚实用户的平均资源
        for j in range(0,54):
            if j!=index_robust1[0]:#诚实用户
                sum_honest_resource_quantity_robust1=sum_honest_resource_quantity_robust1+agent_resource_quantity_robust1[j][k]        
        average_honest_resource_quantity_robust1.append(sum_honest_resource_quantity_robust1/(54-number_robust1))
        
        #输出第k次迭代中的结果
        print(k,"次迭代中的结果：")
        for i in range (0,54):
            print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust1[i][k])
        print("资源申请量真实平均值avg为:",average_resource_quantity_true_robust1[k])
        print("****************************************************************************")
        print("平均资源追踪量的平均值avg为:",x_average_robust1[k])
        for i in range(0,54):
            print("agent",(i+1),"在",k,"次迭代中的的追踪平均量为:",x_average_resource_quantity_robust1[i][k])
        for i in range(0,54):
            print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust1[i][k])       
        print("****************************************************************************")
    
    return None

In [ ]:
FODAC_PD_DRA_Static_Byzantine_robust1(generator_a_values_robust1,generator_b_values_robust1,generator_c_values_robust1,generator_Pmax_values_robust1,generator_Pmin_values_robust1,agent_resource_quantity_robust1,average_resource_quantity_true_robust1,x_average_resource_quantity_robust1,v_robust1,iteration_update_robust1,dual_variable_robust1,W_robust1,iteration_robust1)

In [ ]:
#数据存储
np.savetxt('./data/05_agent_resource_quantity_robust1.txt',agent_resource_quantity_robust1,fmt='%0.8f')#用户资源
np.savetxt('./data/05_average_cost_honest_robust1.txt',average_cost_honest_robust1,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/05_average_honest_resource_quantity_robust1.txt',average_honest_resource_quantity_robust1,fmt='%0.8f')#诚实用户平均资源

# 绘制实验图

In [ ]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import random
from sympy import *
import math 
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
#数据导入
agent_resource_quantity= np.loadtxt("./data/05_agent_resource_quantity.txt")
agent_resource_quantity_attack1=np.loadtxt("./data/05_agent_resource_quantity_attack1.txt")
agent_resource_quantity_robust1=np.loadtxt("./data/05_agent_resource_quantity_robust1.txt")
average_cost=np.loadtxt("./data/05_average_cost.txt")
average_cost_honest_attack1=np.loadtxt("./data/05_average_cost_honest_attack1.txt")
average_cost_honest_robust1=np.loadtxt("./data/05_average_cost_honest_robust1.txt")
average_resource_quantity_true=np.loadtxt("./data/05_average_resource_quantity_true.txt")
average_honest_resource_quantity_attack1=np.loadtxt("./data/05_average_honest_resource_quantity_attack1.txt")
average_honest_resource_quantity_robust1=np.loadtxt("./data/05_average_honest_resource_quantity_robust1.txt")

In [ ]:
#图1：迭代-资源申请量
figsize = 8,6

iteration=[]
iteration.append(0)
for i in range(0,1000):
    iteration.append(i)
agent_resource_quantity_space=[]
for i in range (0,20):
    agent_resource_quantity_space.append(agent_resource_quantity[38][999])

#在同一幅图片上画3条折线
A,=plt.plot(iteration[slice(0,1000,50)],agent_resource_quantity_space,marker = '>',color = 'darkorange',linewidth = '1.5',linestyle = '--',label='Oracle')
B,=plt.plot(iteration[slice(0,1000,50)],agent_resource_quantity_attack1[38][slice(1,1001,50)],marker = '*',color = 'r',linewidth = '1.5',label='DRA')
C,=plt.plot(iteration[slice(0,1000,50)],agent_resource_quantity_robust1[38][slice(1,1001,50)],marker = 's',color = 'g',linewidth = '1.5',label='BREDA')

#设置图例并且设置图例的字体及大小
font1 = {
'size'   : 18,
'family': 'Times New Roman'
}
legend = plt.legend(handles=[A,B,C],prop=font1)
 
#设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=18)
 
#设置横纵坐标的名称以及对应字体格式
font2 = {
'size'   : 18,
'family': 'Times New Roman'
}
plt.xlabel('Number of iterations',font2)
plt.ylabel(r'Resource allocation $P_{i}^{k}$ $(i \in \mathcal{H})$',font2)
plt.title('Max value attack',font2) 
 
#将文件保存至文件中并且画出图

#plt.savefig('./figures/figure5_max_resource.pdf',format='pdf', bbox_inches='tight')

In [ ]:
#图2：迭代-平均成本
figsize = 8,6

fig,ax = plt.subplots()
 
average_cost_space=[]
for i in range (0,20):
    average_cost_space.append(average_cost[999])

#在同一幅图片上画3条折线
A,=plt.plot(iteration[slice(0,1000,50)],average_cost_space,marker = '>',color = 'darkorange',linewidth = '1.5',linestyle = '--',label='Oracle')
B,=plt.plot(iteration[slice(0,1000,50)],average_cost_honest_attack1[slice(0,1000,50)],marker = '*',color = 'r',linewidth = '1.5',label='DRA')
C,=plt.plot(iteration[slice(0,1000,50)],average_cost_honest_robust1[slice(0,1000,50)],marker = 's',color = 'g',linewidth = '1.5',label='BREDA')

#设置图例并且设置图例的字体及大小
font1 = {
'size'   : 18,
'family': 'Times New Roman'
}
legend = plt.legend(handles=[A,B,C],prop=font1)
 
#设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=18)

 
#设置横纵坐标的名称以及对应字体格式
font2 = {
'size'   : 18,
'family': 'Times New Roman'
}

plt.xlabel('Number of iterations',font2)
plt.ylabel(r'Average cost of honest agents $f(P^{k})$',font2)
plt.title('Max value attack',font2) 

plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
ax.yaxis.get_offset_text().set_font_properties('Times New Roman')
ax.yaxis.get_offset_text().set_fontsize(18)
#plt.text(-50,345000, r'x 10$^{5}$',font2)

 
#将文件保存至文件中并且画出图
#plt.savefig('./figures/figure2_uniform_high_max_average_cost.pdf',format='pdf', bbox_inches='tight')

In [ ]:
#图3：迭代-(诚实用户的平均资源量与平均资源阈值s的gap）
figsize = 8,6

average_resource_space=[]
for i in range(0,1000):
    average_resource_space.append((6000/54))
    
average_cost_gap_space=[]
for i in range (0,20):
    average_cost_gap_space.append(average_resource_quantity_true[999]-(6000/54))
    
    
#在同一幅图片上画3条折线
A,=plt.plot(iteration[slice(0,1000,50)],average_cost_gap_space,marker = '>',color = 'darkorange',linewidth = '1.5',linestyle = '--',label='Oracle')
B,=plt.plot(iteration[slice(0,1000,50)],list(map(lambda x: x[0]-x[1], zip(average_honest_resource_quantity_attack1, average_resource_space)))[slice(0,1000,50)],marker = '*',color = 'r',linewidth = '1.5',label='DRA')
C,=plt.plot(iteration[slice(0,1000,50)],list(map(lambda x: x[0]-x[1], zip(average_honest_resource_quantity_robust1, average_resource_space)))[slice(0,1000,50)],marker = 's',color = 'g',linewidth = '1.5',label='BREDA')


#设置图例并且设置图例的字体及大小
font1 = {
'size'   : 18,
'family': 'Times New Roman'
}
legend = plt.legend(handles=[A,B,C],prop=font1)
 
#设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=18)

 
#设置横纵坐标的名称以及对应字体格式
font2 = {
'size'   : 18,
'family': 'Times New Roman'
}
plt.xlabel('Number of iterations',font2)
plt.ylabel(r'$\frac{1}{\left |  \mathcal{H}  \right | } \sum_{i \in \mathcal{H}} P_{i}^{k}-s$',font2)
plt.title('Max value attack',font2) 
 
#将文件保存至文件中并且画出图
#plt.savefig('./figures/figure3_uniform_high_max_average_resource_gap.pdf',format='pdf', bbox_inches='tight')


# DRA算法+截断式高斯攻击

In [ ]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 

In [ ]:
#定义变量 
agent_resource_quantity_attack2 = np.zeros(shape=(54,2000)) 
average_resource_quantity_true_attack2= [] 
x_average_resource_quantity_attack2=np.zeros(shape=(54,2000)) 
v_attack2=0
iteration_update_attack2 = 0.5
dual_variable_attack2 = np.zeros(shape=(54,2000))#对偶变量
generator_a_values_attack2= np.loadtxt("generator_a_values.txt")#每个agent的成本函数中的超参数a
generator_b_values_attack2= np.loadtxt("generator_b_values.txt")#每个agent的成本函数中的超参数a
generator_c_values_attack2= np.loadtxt("generator_c_values.txt")#每个agent的成本函数中的超参数a
generator_Pmax_values_attack2= np.loadtxt("generator_Pmax_values.txt")#每个agent的资源量的上界
generator_Pmin_values_attack2= np.loadtxt("generator_Pmin_values.txt")#每个agent的资源量的下界
W_attack2=np.loadtxt("Adjacency_Matrix_generator_doubly_stochastic.txt") #双随机的网络拓扑矩阵W（对角线处非零）
number_attack2 = 1 #攻击者数量
index_attack2=[27]#攻击者的索引 下标
sum_attack2 = 0 #过渡变量
iteration_attack2=[]#绘图横坐标
average_cost_attack2=np.zeros(shape=(1,2000))#存放每一次迭代后所有用户成本的平均值
average_cost_honest_attack2=np.zeros(shape=(1,2000))#存放每一次迭代后所有诚实用户成本的平均值
average_honest_resource_quantity_attack2=[]#存放每一次迭代后诚实用户的平均资源量
x_average_attack2=[]#存放每一次迭代后所有用户追踪平均值x的平均值
random_attack=np.zeros(shape=(1,2000))#记录n=6个攻击者的攻击资源量

In [ ]:
#变量初始化
#用户资源量的初始化
for i in range(0,54):
    agent_resource_quantity_attack2[i][0]=50
    
#设置攻击者的初始攻击
agent_resource_quantity_attack2[index_attack2[0]][0]=150
        
#追踪均值变量的初始化 x(0)=agent_resource_quantity(0)
for i in range (0,54):
    x_average_resource_quantity_attack2[i][0]=agent_resource_quantity_attack2[i][0]
    
#资源量真实均值的计算 初始值
sum1_attack2=0
for i in range (0,54):
    sum1_attack2 = sum1_attack2 + agent_resource_quantity_attack2[i][0] 
average_resource_quantity_true_attack2.append(sum1_attack2/54)

#平均追踪值x的平均值 初始值
sum2_attack2=0
for i in range (0,54):
    sum2_attack2 = sum2_attack2 + x_average_resource_quantity_attack2[i][0] 
x_average_attack2.append(sum2_attack2/54)

#平均成本 初始值
sum_average_cost_attack2=0
for j in range(0,54):
    sum_average_cost_attack2=sum_average_cost_attack2+(generator_a_values_attack2[j]+(generator_b_values_attack2[j]*agent_resource_quantity_attack2[j][0])+(generator_b_values_attack2[j]*agent_resource_quantity_attack2[j][0]*agent_resource_quantity_attack2[j][0]))
average_cost_attack2[0][0]=sum_average_cost_attack2/54
    
#对偶变量的初始化
for i in range (0,54):
    dual_variable_attack2[i][0]=1

iteration_attack2.append(0)

#计算诚实用户的平均成本 初始值 
sum_average_cost_honest_attack2=0
for j in range(0,54):
    if j!=index_attack2[0]:
        sum_average_cost_honest_attack2=(sum_average_cost_honest_attack2
    +(generator_a_values_attack2[j]+(generator_b_values_attack2[j]*agent_resource_quantity_attack2[j][0])+(generator_b_values_attack2[j]*agent_resource_quantity_attack2[j][0]*agent_resource_quantity_attack2[j][0])))        
average_cost_honest_attack2[0][0]=sum_average_cost_honest_attack2/(54-number_attack2)

#诚实用户的平均资源量
sum_honest_resource_quantity_attack2=0
for j in range(0,54):
    if j!=index_attack2[0]:
        sum_honest_resource_quantity_attack2=sum_honest_resource_quantity_attack2+agent_resource_quantity_attack2[j][0]
average_honest_resource_quantity_attack2.append(sum_honest_resource_quantity_attack2/(54-number_attack2))

In [ ]:
def FODAC_PD_DRA_Static_Byzantine_attack2(generator_a_values_attack2,generator_b_values_attack2,generator_c_values_attack2,generator_Pmax_values_attack2,generator_Pmin_values_attack2,agent_resource_quantity_attack2,average_resource_quantity_true_attack2,x_average_resource_quantity_attack2,v_attack2,iteration_update_attack2,dual_variable_attack2,W_attack2,iteration_attack2):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DRA算法+截断式拜占庭拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,54):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_attack2[i][0])
    print("*************************")
    print("初始资源申请量真实平均值avg为:",average_resource_quantity_true_attack2[0])
    print("*************************")
    print("平均资源追踪量的初始平均值avg为:",x_average_attack2[0])
    print("*************************")
    for i in range(0,54):
        print("agent",(i+1),"的初始追踪平均量为:",x_average_resource_quantity_attack2[i][0])
    print("*************************")
    for i in range(0,54):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_attack2[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_attack2)
    print("****************************************************************************************************")
    
    #开始迭代
    for k in range(1,2000):#迭代次数
        sum1_attack2 = 0
        sum2_attack2 = 0
        sum_average_cost_attack2=0
        sum_average_cost_honest_attack2=0
        iteration_attack2.append(k)
        sum_honest_resource_quantity_attack2=0
        
        #计算诚实用户和攻击用户的资源申请量迭代
        for i in range (0,54):
            if i!=index_attack2[0]:#诚实用户
                agent_resource_quantity_attack2[i][k]=(agent_resource_quantity_attack2[i][k-1]-iteration_update_attack2
            *(((generator_b_values_attack2[i]+2*generator_c_values_attack2[i]*agent_resource_quantity_attack2[i][k-1])/54)+(dual_variable_attack2[i][k-1]/54)))
                #原变量的投影操作
                if agent_resource_quantity_attack2[i][k] < generator_Pmin_values_attack2[i]:
                    agent_resource_quantity_attack2[i][k]= generator_Pmin_values_attack2[i]
                elif agent_resource_quantity_attack2[i][k] > generator_Pmax_values_attack2[i]:
                    agent_resource_quantity_attack2[i][k]=generator_Pmax_values_attack2[i]
            else:#攻击用户
                agent_resource_quantity_attack2[i][k]=np.random.normal(150, 30, 1)#截断式高斯攻击
                if agent_resource_quantity_attack2[i][k] < generator_Pmin_values_attack2[i]:
                    agent_resource_quantity_attack2[i][k]= generator_Pmin_values_attack2[i]
                elif agent_resource_quantity_attack2[i][k] > generator_Pmax_values_attack2[i]:
                    agent_resource_quantity_attack2[i][k]=generator_Pmax_values_attack2[i]
                random_attack[0][k]=agent_resource_quantity_attack2[i][k]
            
        #计算诚实用户和攻击用户的平均值追踪量的迭代 Wx+资源量差值
        for i in range (0,54):
            if i!=index_attack2[0]:#诚实用户
                sum_x_attack2=0
                for j in range (0,54):
                    if W_attack2[i][j]!=0:
                        sum_x_attack2=sum_x_attack2+W_attack2[i][j]*x_average_resource_quantity_attack2[j][k-1]
                x_average_resource_quantity_attack2[i][k]=sum_x_attack2+(agent_resource_quantity_attack2[i][k]-agent_resource_quantity_attack2[i][k-1])                     
            else:#攻击者
                x_average_resource_quantity_attack2[i][k]=agent_resource_quantity_attack2[i][k-1] 
        
        #诚实用户和攻击者对偶变量的迭代
        for i in range(0,54):
            if i!=index_attack2[0]:#诚实用户
                dual_variable_attack2[i][k]=(dual_variable_attack2[i][k-1]+iteration_update_attack2
                *((x_average_resource_quantity_attack2[i][k-1])-(6000/54)-(v_attack2*dual_variable_attack2[i][k-1])))
            else:#攻击者
                dual_variable_attack2[i][k]=dual_variable_attack2[i][k-1] #被攻击用户的对偶变量不进行更新
            
        #对偶变量的正投影操作
        '''for i in range(0,54):
            if dual_variable_attack2[i][k] < 0:
                dual_variable_attack2[i][k] =0 '''
                
        #计算所有用户平均资源量
        for i in range (0,54):
            sum1_attack2 = sum1_attack2 + agent_resource_quantity_attack2[i][k]
        average_resource_quantity_true_attack2.append(sum1_attack2/54)
        
        #计算平均资源追踪值x的平均值
        for i in range (0,54):
            sum2_attack2 = sum2_attack2 + x_average_resource_quantity_attack2[i][k]
        x_average_attack2.append(sum2_attack2/54)
        
        #计算所有用户平均成本
        for j in range(0,54):
            sum_average_cost_attack2=(sum_average_cost_attack2+
            (generator_a_values_attack2[j]+(generator_b_values_attack2[j]*agent_resource_quantity_attack2[j][k])
             +(generator_b_values_attack2[j]*agent_resource_quantity_attack2[j][k]*agent_resource_quantity_attack2[j][k])))
        average_cost_attack2[0][k]=sum_average_cost_attack2/54

        #计算诚实用户的平均成本 
        for j in range(0,54):
            if j!=index_attack2[0]:#诚实用户
                sum_average_cost_honest_attack2=(sum_average_cost_honest_attack2+(generator_a_values_attack2[j]+(generator_b_values_attack2[j]*agent_resource_quantity_attack2[j][k])+(generator_b_values_attack2[j]*agent_resource_quantity_attack2[j][k]*agent_resource_quantity_attack2[j][k])))    
        average_cost_honest_attack2[0][k]=sum_average_cost_honest_attack2/(54-number_attack2)
        
        #计算诚实用户的平均资源
        for j in range(0,54):
            if j!=index_attack2[0]:#诚实用户
                sum_honest_resource_quantity_attack2=sum_honest_resource_quantity_attack2+agent_resource_quantity_attack2[j][k]        
        average_honest_resource_quantity_attack2.append(sum_honest_resource_quantity_attack2/(54-number_attack2))
        
        #输出第k次迭代中的结果
        print(k,"次迭代中的结果：")
        for i in range (0,54):
            print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_attack2[i][k])
        print("资源申请量真实平均值avg为:",average_resource_quantity_true_attack2[k])
        print("****************************************************************************")
        print("平均资源追踪量的平均值avg为:",x_average_attack2[k])
        for i in range(0,54):
            print("agent",(i+1),"在",k,"次迭代中的的追踪平均量为:",x_average_resource_quantity_attack2[i][k])
        for i in range(0,54):
            print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_attack2[i][k])       
        print("****************************************************************************")
    
    return None

In [ ]:
FODAC_PD_DRA_Static_Byzantine_attack2(generator_a_values_attack2,generator_b_values_attack2,generator_c_values_attack2,generator_Pmax_values_attack2,generator_Pmin_values_attack2,agent_resource_quantity_attack2,average_resource_quantity_true_attack2,x_average_resource_quantity_attack2,v_attack2,iteration_update_attack2,dual_variable_attack2,W_attack2,iteration_attack2)

In [ ]:
#数据存储
np.savetxt('./data/05_agent_resource_quantity_attack2.txt',agent_resource_quantity_attack2,fmt='%0.8f')#用户资源
np.savetxt('./data/05_average_cost_honest_attack2.txt',average_cost_honest_attack2,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/05_average_honest_resource_quantity_attack2.txt',average_honest_resource_quantity_attack2,fmt='%0.8f')#诚实用户平均资源

# BREDA算法+截断式高斯攻击

In [ ]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 

In [ ]:
#定义变量 
agent_resource_quantity_robust2= np.zeros(shape=(54,1000)) 
average_resource_quantity_true_robust2= [] 
x_average_resource_quantity_robust2=np.zeros(shape=(54,1000)) 
v_robust2=0
iteration_update_robust2 = 0.5
dual_variable_robust2 = np.zeros(shape=(54,1000))#对偶变量
generator_a_values_robust2= np.loadtxt("generator_a_values.txt")#每个agent的成本函数中的超参数a
generator_b_values_robust2= np.loadtxt("generator_b_values.txt")#每个agent的成本函数中的超参数a
generator_c_values_robust2= np.loadtxt("generator_c_values.txt")#每个agent的成本函数中的超参数a
generator_Pmax_values_robust2= np.loadtxt("generator_Pmax_values.txt")#每个agent的资源量的上界
generator_Pmin_values_robust2= np.loadtxt("generator_Pmin_values.txt")#每个agent的资源量的下界
W_robust2=np.loadtxt("Adjacency_Matrix_generator_doubly_stochastic.txt") #双随机的网络拓扑矩阵W（对角线处非零）
A_robust2=np.loadtxt("Adjacency_Matrix_generator_zero.txt") #网络拓扑邻接矩阵（对角线处为0）
D_robust2=np.loadtxt("degree_generator.txt") #用户的度
number_robust2 = 1 #攻击者数量
index_robust2=[27]#攻击者的索引 下标
sum_robust2 = 0 #过渡变量
iteration_robust2=[]#绘图横坐标
average_cost_robust2=np.zeros(shape=(1,1000))#存放每一次迭代后所有用户成本的平均值
average_cost_honest_robust2=np.zeros(shape=(1,1000))#存放每一次迭代后所有诚实用户成本的平均值
average_honest_resource_quantity_robust2=[]#存放每一次迭代后诚实用户的平均资源量
x_average_robust2=[]#存放每一次迭代后所有用户追踪平均值x的平均值
neighbor_robust2=np.zeros(shape=(54,16))#定义矩阵存放54个用户的邻居
neighbor_robust2_help=[]#辅助求用户邻居的列表

In [ ]:
#变量初始化
#用户资源量的初始化
for i in range(0,54):
    agent_resource_quantity_robust2[i][0]=50
    
#设置攻击者的初始攻击
agent_resource_quantity_robust2[index_robust2[0]][0]=300
        
#追踪均值变量的初始化 x(0)=agent_resource_quantity(0)
for i in range (0,54):
    x_average_resource_quantity_robust2[i][0]=agent_resource_quantity_robust2[i][0]
    
#资源量真实均值的计算 初始值
sum1_robust2=0
for i in range (0,54):
    sum1_robust2 = sum1_robust2 + agent_resource_quantity_robust2[i][0] 
average_resource_quantity_true_robust2.append(sum1_robust2/54)

#平均追踪值x的平均值 初始值
sum2_robust2=0
for i in range (0,54):
    sum2_robust2 = sum2_robust2 + x_average_resource_quantity_robust2[i][0] 
x_average_robust2.append(sum2_robust2/54)

#平均成本 初始值
sum_average_cost_robust2=0
for j in range(0,54):
    sum_average_cost_robust2=sum_average_cost_robust2+(generator_a_values_robust2[j]+(generator_b_values_robust2[j]*agent_resource_quantity_robust2[j][0])+(generator_b_values_robust2[j]*agent_resource_quantity_robust2[j][0]*agent_resource_quantity_robust2[j][0]))
average_cost_robust2[0][0]=sum_average_cost_robust2/54
    
#对偶变量的初始化
for i in range (0,54):
    dual_variable_robust2[i][0]=1

iteration_robust2.append(0)

#计算诚实用户的平均成本 初始值 
sum_average_cost_honest_robust2=0
for j in range(0,54):
    if j!=index_robust2[0]:
        sum_average_cost_honest_robust2=(sum_average_cost_honest_robust2
    +(generator_a_values_robust2[j]+(generator_b_values_robust2[j]*agent_resource_quantity_robust2[j][0])+(generator_b_values_robust2[j]*agent_resource_quantity_robust2[j][0]*agent_resource_quantity_robust2[j][0])))        
average_cost_honest_robust2[0][0]=sum_average_cost_honest_robust2/(54-number_robust2)

#诚实用户的平均资源量
sum_honest_resource_quantity_robust2=0
for j in range(0,54):
    if j!=index_robust2[0]:
        sum_honest_resource_quantity_robust2=sum_honest_resource_quantity_robust2+agent_resource_quantity_robust2[j][0]
average_honest_resource_quantity_robust2.append(sum_honest_resource_quantity_robust2/(54-number_robust2))

#对用户邻居进行初始化
for i in range(0,54):
    for j in range(0,16):
        neighbor_robust2[i][j]=99 #先将矩阵中的所有数字设定为一个不可能的数字
#根据邻接矩阵求每个用户的邻居集合
for i in range(0,54):
    for j in range(0,54):
        if A_robust2[i][j]!=0:
            neighbor_robust2_help.append(j)
k=0
for i in range(0,54):
    g=0
    for j in range(k,k+int(D_robust2[i])):
        neighbor_robust2[i][g]=neighbor_robust2_help[j]  
        g=g+1
    k=k+int(D_robust2[i])    

In [ ]:
#BREDA算法 用TM进行x的鲁棒聚合
def FODAC_PD_DRA_Static_Byzantine_robust2(generator_a_values_robust2,generator_b_values_robust2,generator_c_values_robust2,generator_Pmax_values_robust2,generator_Pmin_values_robust2,agent_resource_quantity_robust2,average_resource_quantity_true_robust2,x_average_resource_quantity_robust2,v_robust2,iteration_update_robust2,dual_variable_robust2,W_robust2,iteration_robust2):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************BREDA算法+截断式高斯拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,54):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust2[i][0])
    print("*************************")
    print("初始资源申请量真实平均值avg为:",average_resource_quantity_true_robust2[0])
    print("*************************")
    print("平均资源追踪量的初始平均值avg为:",x_average_robust2[0])
    print("*************************")
    for i in range(0,54):
        print("agent",(i+1),"的初始追踪平均量为:",x_average_resource_quantity_robust2[i][0])
    print("*************************")
    for i in range(0,54):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust2[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust2)
    print("****************************************************************************************************")
    
    #开始迭代
    for k in range(1,1000):#迭代次数
        sum1_robust2 = 0
        sum2_robust2 = 0
        sum_average_cost_robust2=0
        sum_average_cost_honest_robust2=0
        iteration_robust2.append(k)
        sum_honest_resource_quantity_robust2=0
        
        #计算诚实用户和攻击用户的资源申请量迭代
        for i in range (0,54):
            if i!=index_robust2[0]:#诚实用户
                agent_resource_quantity_robust2[i][k]=(agent_resource_quantity_robust2[i][k-1]-iteration_update_robust2
            *(((generator_b_values_robust2[i]+2*generator_c_values_robust2[i]*agent_resource_quantity_robust2[i][k-1])/54)+(dual_variable_robust2[i][k-1]/54)))
                #原变量的投影操作
                if agent_resource_quantity_robust2[i][k] < generator_Pmin_values_robust2[i]:
                    agent_resource_quantity_robust2[i][k]= generator_Pmin_values_robust2[i]
                elif agent_resource_quantity_robust2[i][k] > generator_Pmax_values_robust2[i]:
                    agent_resource_quantity_robust2[i][k]=generator_Pmax_values_robust2[i]
            else:#攻击用户
                agent_resource_quantity_robust2[i][k]=random_attack[0][k]
            
        #计算诚实用户和攻击用户的平均值追踪量的迭代 Wx+资源量差值
        #用trimmed-mean算法对诚实用户的x进行鲁棒聚合
        for i in range (0,54):
            if i!=index_robust2[0]:#诚实用户
                sort_list_robust2=[]
                sum_x_robust2=0
                for j in range(0,int(D_robust2[i])):
                    sort_list_robust2.append(x_average_resource_quantity_robust2[int(neighbor_robust2[i][j])][k-1])
                sort_list_robust2.sort()
                #剔除一个最大的数据和一个最小的数据
                if int(D_robust2[i])>2: #度大于2的用户
                    del sort_list_robust2[int(D_robust2[i])-1]
                    del sort_list_robust2[0]
                    for j in range(0,len(sort_list_robust2)):
                        sum_x_robust2=sum_x_robust2+sort_list_robust2[j]
                    x_average_resource_quantity_robust2[i][k]=((1/(len(sort_list_robust2)+1))*(x_average_resource_quantity_robust2[i][k-1]+sum_x_robust2)
                    +(agent_resource_quantity_robust2[i][k]-agent_resource_quantity_robust2[i][k-1]))
                if int(D_robust2[i])==2: #对于度小于等于2的用户 不进行TM操作 直接用WX
                    for j in range(0,2):
                        sum_x_robust2=sum_x_robust2+(W_robust2[i][int(neighbor_robust2[i][j])]*x_average_resource_quantity_robust2[int(neighbor_robust2[i][j])][k-1])                       
                    x_average_resource_quantity_robust2[i][k]=(W_robust2[i][i]*x_average_resource_quantity_robust2[i][k-1]+sum_x_robust2
                    +(agent_resource_quantity_robust2[i][k]-agent_resource_quantity_robust2[i][k-1]))
                if int(D_robust2[i])==1:
                    x_average_resource_quantity_robust2[i][k]=(W_robust2[i][i]*x_average_resource_quantity_robust2[i][k-1]+W_robust2[i][int(neighbor_robust2[i][j])]*x_average_resource_quantity_robust2[int(neighbor_robust2[i][0])][k-1]
                    +(agent_resource_quantity_robust2[i][k]-agent_resource_quantity_robust2[i][k-1]))                                
            else:#攻击者
                x_average_resource_quantity_robust2[i][k]=agent_resource_quantity_robust2[i][k-1]
        
        #诚实用户和攻击者对偶变量的迭代
        for i in range(0,54):
            if i!=index_robust2[0]:#诚实用户
                dual_variable_robust2[i][k]=(dual_variable_robust2[i][k-1]+iteration_update_robust2
                *((x_average_resource_quantity_robust2[i][k-1])-(6000/54)-(v_robust2*dual_variable_robust2[i][k-1])))
            else:#攻击者
                dual_variable_robust2[i][k]=dual_variable_robust2[i][k-1] #被攻击用户的对偶变量不进行更新
            
        #对偶变量的正投影操作
        '''for i in range(0,54):
            if dual_variable_robust2[i][k] < 0:
                dual_variable_robust2[i][k] =0 '''
                
        #计算所有用户平均资源量
        for i in range (0,54):
            sum1_robust2 = sum1_robust2 + agent_resource_quantity_robust2[i][k]
        average_resource_quantity_true_robust2.append(sum1_robust2/54)
        
        #计算平均资源追踪值x的平均值
        for i in range (0,54):
            sum2_robust2 = sum2_robust2 + x_average_resource_quantity_robust2[i][k]
        x_average_robust2.append(sum2_robust2/54)
        
        #计算所有用户平均成本
        for j in range(0,54):
            sum_average_cost_robust2=(sum_average_cost_robust2+
            (generator_a_values_robust2[j]+(generator_b_values_robust2[j]*agent_resource_quantity_robust2[j][k])
             +(generator_b_values_robust2[j]*agent_resource_quantity_robust2[j][k]*agent_resource_quantity_robust2[j][k])))
        average_cost_robust2[0][k]=sum_average_cost_robust2/54

        #计算诚实用户的平均成本 
        for j in range(0,54):
            if j!=index_robust2[0]:#诚实用户
                sum_average_cost_honest_robust2=(sum_average_cost_honest_robust2+(generator_a_values_robust2[j]+(generator_b_values_robust2[j]*agent_resource_quantity_robust2[j][k])+(generator_b_values_robust2[j]*agent_resource_quantity_robust2[j][k]*agent_resource_quantity_robust2[j][k])))    
        average_cost_honest_robust2[0][k]=sum_average_cost_honest_robust2/(54-number_robust2)
        
        #计算诚实用户的平均资源
        for j in range(0,54):
            if j!=index_robust2[0]:#诚实用户
                sum_honest_resource_quantity_robust2=sum_honest_resource_quantity_robust2+agent_resource_quantity_robust2[j][k]        
        average_honest_resource_quantity_robust2.append(sum_honest_resource_quantity_robust2/(54-number_robust2))
        
        #输出第k次迭代中的结果
        print(k,"次迭代中的结果：")
        for i in range (0,54):
            print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust2[i][k])
        print("资源申请量真实平均值avg为:",average_resource_quantity_true_robust2[k])
        print("****************************************************************************")
        print("平均资源追踪量的平均值avg为:",x_average_robust2[k])
        for i in range(0,54):
            print("agent",(i+1),"在",k,"次迭代中的的追踪平均量为:",x_average_resource_quantity_robust2[i][k])
        for i in range(0,54):
            print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust2[i][k])       
        print("****************************************************************************")
    
    return None

In [ ]:
FODAC_PD_DRA_Static_Byzantine_robust2(generator_a_values_robust2,generator_b_values_robust2,generator_c_values_robust2,generator_Pmax_values_robust2,generator_Pmin_values_robust2,agent_resource_quantity_robust2,average_resource_quantity_true_robust2,x_average_resource_quantity_robust2,v_robust2,iteration_update_robust2,dual_variable_robust2,W_robust2,iteration_robust2)

In [ ]:
#数据存储
np.savetxt('./data/05_agent_resource_quantity_robust2.txt',agent_resource_quantity_robust2,fmt='%0.8f')#用户资源
np.savetxt('./data/05_average_cost_honest_robust2.txt',average_cost_honest_robust2,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/05_average_honest_resource_quantity_robust2.txt',average_honest_resource_quantity_robust2,fmt='%0.8f')#诚实用户平均资源

# 绘制实验图

In [ ]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import random
from sympy import *
import math 
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
agent_resource_quantity= np.loadtxt("./data/05_agent_resource_quantity.txt")
agent_resource_quantity_attack1=np.loadtxt("./data/05_agent_resource_quantity_attack1.txt")
agent_resource_quantity_robust1=np.loadtxt("./data/05_agent_resource_quantity_robust1.txt")
average_cost=np.loadtxt("./data/05_average_cost.txt")
average_cost_honest_attack1=np.loadtxt("./data/05_average_cost_honest_attack1.txt")
average_cost_honest_robust1=np.loadtxt("./data/05_average_cost_honest_robust1.txt")
average_resource_quantity_true=np.loadtxt("./data/05_average_resource_quantity_true.txt")
average_honest_resource_quantity_attack1=np.loadtxt("./data/05_average_honest_resource_quantity_attack1.txt")
average_honest_resource_quantity_robust1=np.loadtxt("./data/05_average_honest_resource_quantity_robust1.txt")
agent_resource_quantity_attack2=np.loadtxt("./data/05_agent_resource_quantity_attack2.txt")
agent_resource_quantity_robust2=np.loadtxt("./data/05_agent_resource_quantity_robust2.txt")
average_cost_honest_attack2=np.loadtxt("./data/05_average_cost_honest_attack2.txt")
average_cost_honest_robust2=np.loadtxt("./data/05_average_cost_honest_robust2.txt")
average_honest_resource_quantity_attack2=np.loadtxt("./data/05_average_honest_resource_quantity_attack2.txt")
average_honest_resource_quantity_robust2=np.loadtxt("./data/05_average_honest_resource_quantity_robust2.txt")

In [ ]:

plt.subplot(2, 3, 1)##############################################################


iteration=[]
iteration.append(0)
for i in range(0,1000):
    iteration.append(i)

agent_resource_quantity_space=[]
for i in range (0,20):
    agent_resource_quantity_space.append(agent_resource_quantity[38][999])

#在同一幅图片上画3条折线
A,=plt.plot(iteration[slice(0,1000,50)],agent_resource_quantity_space,marker = '>',color = 'darkorange',linewidth = '1.5',linestyle = '--',label='Oracle')
B,=plt.plot(iteration[slice(0,1000,50)],agent_resource_quantity_attack1[38][slice(1,1001,50)],marker = '*',color = 'r',linewidth = '1.5',label='DRA')
C,=plt.plot(iteration[slice(0,1000,50)],agent_resource_quantity_robust1[38][slice(1,1001,50)],marker = 's',color = 'g',linewidth = '1.5',label='BREDA')

#设置图例并且设置图例的字体及大小
font1 = {
'size'   : 18,
'family': 'Times New Roman'
}
legend = plt.legend(handles=[A,B,C],prop=font1)
 
#设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=18)
 
#设置横纵坐标的名称以及对应字体格式
font2 = {
'size'   : 18,
'family': 'Times New Roman'
}
plt.xlabel('Number of iterations',font2)
plt.ylabel(r'Resource allocation $\theta_{i}^{k}$ $(i \in \mathcal{H})$',font2)
plt.title('Max value attack',font2) 


#fig,ax = plt.subplot(2,3,2)
plt.subplot(2, 3, 2)#########################################################################

average_cost_space=[]
for i in range (0,20):
    average_cost_space.append(average_cost[999])

#在同一幅图片上画3条折线
A,=plt.plot(iteration[slice(0,1000,50)],average_cost_space,marker = '>',color = 'darkorange',linewidth = '1.5',linestyle = '--',label='Oracle')
B,=plt.plot(iteration[slice(0,1000,50)],average_cost_honest_attack1[slice(0,1000,50)],marker = '*',color = 'r',linewidth = '1.5',label='DRA')
C,=plt.plot(iteration[slice(0,1000,50)],average_cost_honest_robust1[slice(0,1000,50)],marker = 's',color = 'g',linewidth = '1.5',label='BREDA')

#设置图例并且设置图例的字体及大小
font1 = {
'size'   : 18,
'family': 'Times New Roman'
}
legend = plt.legend(handles=[A,B,C],prop=font1)
 
#设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=18)

 
#设置横纵坐标的名称以及对应字体格式
font2 = {
'size'   : 18,
'family': 'Times New Roman'
}
plt.xlabel('Number of iterations',font2)
plt.ylabel(r'Average cost of honest agents $f(\theta^{k})$',font2)
plt.title('Max value attack',font2) 

#plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
'''ax.yaxis.get_offset_text().set_font_properties('Times New Roman')
ax.yaxis.get_offset_text().set_fontsize(18)'''

plt.subplot(2, 3, 3)#########################################################################


average_resource_space=[]
for i in range(0,1000):
    average_resource_space.append(6000/54)
    
average_cost_gap_space=[]
for i in range (0,20):
    average_cost_gap_space.append(average_resource_quantity_true[999]-(6000/54))
    
    
#在同一幅图片上画3条折线
A,=plt.plot(iteration[slice(0,1000,50)],average_cost_gap_space,marker = '>',color = 'darkorange',linewidth = '1.5',linestyle = '--',label='Oracle')
B,=plt.plot(iteration[slice(0,1000,50)],list(map(lambda x: x[0]-x[1], zip(average_honest_resource_quantity_attack1, average_resource_space)))[slice(0,1000,50)],marker = '*',color = 'r',linewidth = '1.5',label='DRA')
C,=plt.plot(iteration[slice(0,1000,50)],list(map(lambda x: x[0]-x[1], zip(average_honest_resource_quantity_robust1, average_resource_space)))[slice(0,1000,50)],marker = 's',color = 'g',linewidth = '1.5',label='BREDA')


#设置图例并且设置图例的字体及大小
font1 = {
'size'   : 18,
'family': 'Times New Roman'
}
legend = plt.legend(handles=[A,B,C],prop=font1)
 
#设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=18)

 
#设置横纵坐标的名称以及对应字体格式
font2 = {
'size'   : 18,
'family': 'Times New Roman'
}
plt.xlabel('Number of iterations',font2)
plt.ylabel(r'$\frac{1}{\left |  \mathcal{H}  \right | } \sum_{i \in \mathcal{H}} \theta_{i}^{k}-s$',font2)
plt.title('Max value attack',font2) 
 
#将文件保存至文件中并且画出图
#plt.savefig('./figures/figure3_uniform_high_max_average_resource_gap.pdf',format='pdf', bbox_inches='tight')


plt.subplot(2, 3, 4)##############################################################################

agent_resource_quantity_space=[]
for i in range (0,20):
    agent_resource_quantity_space.append(agent_resource_quantity[38][999])

#在同一幅图片上画3条折线
A,=plt.plot(iteration[slice(0,1000,50)],agent_resource_quantity_space,marker = '>',color = 'darkorange',linewidth = '1.5',linestyle = '--',label='Oracle')
B,=plt.plot(iteration[slice(0,1000,50)],agent_resource_quantity_attack2[38][slice(1,1001,50)],marker = '*',color = 'r',linewidth = '1.5',label='DRA')
C,=plt.plot(iteration[slice(0,1000,50)],agent_resource_quantity_robust2[38][slice(1,1001,50)],marker = 's',color = 'g',linewidth = '1.5',label='BREDA')

#设置图例并且设置图例的字体及大小
font1 = {
'size'   : 18,
'family': 'Times New Roman'
}
legend = plt.legend(handles=[A,B,C],prop=font1)
 
#设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=18)

 
#设置横纵坐标的名称以及对应字体格式
font2 = {
'size'   : 18,
'family': 'Times New Roman'
}
plt.xlabel('Number of iterations',font2)
plt.ylabel(r'Resource allocation $\theta_{i}^{k}$ $(i \in \mathcal{H})$',font2)
plt.title('Trimmed Gaussian attack',font2) 
 
#将文件保存至文件中并且画出图
#plt.savefig('./figures/figure4_uniform_high_Gaussian_resource.pdf',format='pdf', bbox_inches='tight')


plt.subplot(2, 3, 5)##################################################################################

 
average_cost_space=[]
for i in range (0,20):
    average_cost_space.append(average_cost[999])

#在同一幅图片上画3条折线
A,=plt.plot(iteration[slice(0,1000,50)],average_cost_space,marker = '>',color = 'darkorange',linewidth = '1.5',linestyle = '--',label='Oracle ')
B,=plt.plot(iteration[slice(0,1000,50)],average_cost_honest_attack2[slice(0,1000,50)],marker = '*',color = 'r',linewidth = '1.5',label='DRA')
C,=plt.plot(iteration[slice(0,1000,50)],average_cost_honest_robust2[slice(0,1000,50)],marker = 's',color = 'g',linewidth = '1.5',label='BREDA')

#设置图例并且设置图例的字体及大小
font1 = {
'size'   : 18,
'family': 'Times New Roman'
}
legend = plt.legend(handles=[A,B,C],prop=font1)
 
#设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=18)

 
#设置横纵坐标的名称以及对应字体格式
font2 = {
'size'   : 18,
'family': 'Times New Roman'
}
plt.xlabel('Number of iterations',font2)
plt.ylabel(r'Average cost of honest agents $f(\theta^{k})$',font2)
plt.title('Trimmed Gaussian attack',font2) 
#plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
'''ax.yaxis.get_offset_text().set_font_properties('Times New Roman')
ax.yaxis.get_offset_text().set_fontsize(18)'''
 
#将文件保存至文件中并且画出图
#plt.savefig('./figures/figure5_uniform_high_Gaussian_average_cost.pdf',format='pdf', bbox_inches='tight')


plt.subplot(2, 3, 6)###############################################################################


average_resource_space=[]
for i in range(0,1000):
    average_resource_space.append(6000/54)

average_cost_gap_space=[]
for i in range (0,20):
    average_cost_gap_space.append(average_resource_quantity_true[999]-(6000/54))
     
#在同一幅图片上画3条折线
A,=plt.plot(iteration[slice(0,1000,50)],average_cost_gap_space,marker = '>',color = 'darkorange',linewidth = '1.5',linestyle = '--',label='Oracle')
B,=plt.plot(iteration[slice(0,1000,50)],list(map(lambda x: x[0]-x[1], zip(average_honest_resource_quantity_attack2, average_resource_space)))[slice(0,1000,50)],marker = '*',color = 'r',linewidth = '1.5',label='DRA')
C,=plt.plot(iteration[slice(0,1000,50)],list(map(lambda x: x[0]-x[1], zip(average_honest_resource_quantity_robust2, average_resource_space)))[slice(0,1000,50)],marker = 's',color = 'g',linewidth = '1.5',label='BREDA')

#设置图例并且设置图例的字体及大小
font1 = {
'size'   : 18,
'family': 'Times New Roman'
}
legend = plt.legend(handles=[A,B,C],prop=font1)
 
#设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=18)

 
#设置横纵坐标的名称以及对应字体格式
font2 = {
'size'   : 18,
'family': 'Times New Roman'
}
plt.xlabel('Number of iterations',font2)
plt.ylabel(r'$\frac{1}{\left |  \mathcal{H}  \right | } \sum_{i \in \mathcal{H}} \theta_{i}^{k}-s$',font2)
plt.title('Trimmed Gaussian attack',font2) 
 
#将文件保存至文件中并且画出图

plt.gcf().set_size_inches((24, 12))
plt.subplots_adjust(hspace=0.4, wspace=0.4)


plt.savefig('./figures/figure4.pdf',format='pdf', bbox_inches='tight')

